In [1]:
!wget https://github.com/Phylliida/Dialogue-Datasets/raw/master/MovieCorpus.txt
!python -m nltk.downloader all

--2018-10-11 16:25:07--  https://github.com/Phylliida/Dialogue-Datasets/raw/master/MovieCorpus.txt
Resolving github.com (github.com)... 192.30.253.112, 192.30.253.113
Connecting to github.com (github.com)|192.30.253.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Phylliida/Dialogue-Datasets/master/MovieCorpus.txt [following]
--2018-10-11 16:25:07--  https://raw.githubusercontent.com/Phylliida/Dialogue-Datasets/master/MovieCorpus.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16976724 (16M) [text/plain]
Saving to: ‘MovieCorpus.txt.1’

MovieCorpus.txt.1   100%[===================>]  16.19M  25.8MB/s    in 0.6s    

2018-10-11 16:25:08 (25.8 MB/s) - ‘MovieCorpus.txt.1’ saved [16976724/16976724

In [0]:
import string
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
import numpy as np
from nltk.corpus import stopwords
import warnings
import tqdm
import os
import tensorflow as tf
warnings.filterwarnings('ignore')

In [0]:
print_interval = 1000
window_size = 5
n_epoch = int(1e4)
batch_size = 32

In [4]:

sw = stopwords.words("english")
print(sw)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [0]:

with open("MovieCorpus.txt","r") as f:
  punc_dict = str.maketrans('','',string.punctuation)
  l = f.read().split('\n')
  l = [word.translate(punc_dict) for word in l if not word in sw]
  l = [word.lower() for word in l if word.isalpha()]
  l = sorted(list(set(l)),key=l.index)

In [0]:

le = LabelEncoder().fit(l)
oh = OneHotEncoder().fit(le.transform(l).reshape(len(l),1))
def transform(w):
  return oh.transform(le.transform([w]).reshape(1,1))
def inv_transform(w):
  return  le.inverse_transform(np.argmax(w))

In [7]:
def vectorize(l):
  X = []
  temp = []
  w = 0
  pbar = tqdm.tqdm(total = (len(l)-window_size))
  while w < len(l):
    if len(temp)<window_size:
      temp.append(transform(l[w]))
      w+=1
    else:
      X.append(temp)
      w = w - (window_size - 1)
      temp = []
      pbar.update()
  if len(l) == window_size:
    X.append(temp)
    temp = []
  pbar.close()
  return X
X = vectorize(l)

100%|██████████| 4117/4117 [00:47<00:00, 87.36it/s]


In [0]:
dict_len = len(l)
X = np.array(X)

In [0]:

def gen_weights(shape):
  return tf.Variable(tf.random_normal(shape))
def gen_bias(shape):
  return tf.Variable(tf.random_normal([shape]))
def flatten(in_):
  return tf.reshape(in_,[int(in_.shape[0]),in_.shape[1:].num_elements()])
def dense(in_,out):
  return tf.matmul(in_,gen_weights([int(in_.shape[-1]),out]))+gen_bias(out)

In [0]:
def convert_func(m):
  ret = []
  ret2 = []
  for i in m:
    for j in i: #len(i) == window_size
      ret.append(j.toarray().flatten())
    ret2.append(i[len(i)>>1].toarray().flatten())
  ret = np.array(ret).reshape([len(m),window_size,dict_len])
  ret2 = np.array(ret2).reshape([len(m),dict_len])
  return ret,ret2


In [0]:
x = tf.placeholder(tf.float32,[None,window_size*dict_len])
y = tf.placeholder(tf.float32,[None,dict_len])
def encoder(X,out_l):
  h = tf.nn.relu(dense(X,1024))
 # h = dense(h,128)
  out = dense(h,out_l)
  #h = dense(out,128)
  h = tf.nn.relu(dense(h,1024))
  fin = dense(h,dict_len)
  return fin,out

In [12]:
model,vec = encoder(x,128)
vec = tf.nn.sigmoid(vec)
print(model.shape,vec.shape)

(?, 4122) (?, 128)


In [0]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y,logits = model))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(loss)

In [14]:
sess = tf.Session()
with tf.device('/gpu:0'):
  sess.run(tf.global_variables_initializer())
  for i in range(n_epoch):
      items = np.random.randint(0,len(X)-window_size,batch_size)
      x_input,y_input = convert_func(X[items])
      _,l = sess.run([optimizer,loss], feed_dict={x:x_input.reshape(batch_size,x_input.shape[1]*x_input.shape[2]),y:y_input.reshape(batch_size,y_input.shape[1])})
      if not i%print_interval:
        print("EPOCH:%d Loss:%f"%(i,l))

EPOCH:0 Loss:4284.592285
EPOCH:1000 Loss:2586.159912
EPOCH:2000 Loss:1457.602539
EPOCH:3000 Loss:907.650146
EPOCH:4000 Loss:696.517822
EPOCH:5000 Loss:345.039001
EPOCH:6000 Loss:150.897614
EPOCH:7000 Loss:45.809509
EPOCH:8000 Loss:21.511286
EPOCH:9000 Loss:14.446163


In [15]:
vec_dict = {}
def create_vec_dict():
  for i in tqdm.tqdm(range(len(X))):
    x_input,y_input = convert_func(np.array([X[i]]))
    vec_dict[inv_transform(y_input[0])] = sess.run(vec,feed_dict={x:x_input.reshape(1,x_input.shape[1]*x_input.shape[2])}).flatten()
create_vec_dict()

100%|██████████| 4117/4117 [00:12<00:00, 330.61it/s]


In [0]:
import pickle
with open("file-128.pkl","wb") as f:
  pickle.dump(vec_dict,f)

In [0]:
from google.colab import files
files.download("file-128.pkl")